In [10]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.listdir("../runs/StMoritz/output")

['results_inout.nc', 'results_run.nc']

In [3]:
data_dir = "../runs/StMoritz/output"
fname = "results"
data_file = os.path.join(data_dir, "%s_run.nc" % fname)
inout_file = os.path.join(data_dir, "%s_inout.nc" % fname)

# open without chunks otherwise loading takes forever
data = xr.open_dataset(data_file, decode_times=True)
inout = xr.open_dataset(inout_file, decode_times=True)

In [5]:
# read all the grid- and time-related outputs
time_plt = data.time
xG = data.XG_p1
yG = data.YG_p1
xC = data.XC
yC = data.YC
bathy = data.Depth.to_masked_array()

xx_com = np.array(xG.data)
yy_com = np.array(yG.data)


In [6]:
 # realease depth which will be plotted
z0 = [1,3]
zdata = []
for zz in z0:
    sel_pid = inout.pid.where(inout.z_ini==zz, drop=True)
    zd = data.sel(pid=sel_pid, drop=True)
    zd.xtrack.load()
    zd.ytrack.load()
    zd.ztrack.load()
    zdata.append(zd)

In [7]:
output_path = data_dir+'PT_postproc/'

In [12]:
# plot the snapshots of particle tracking results (will be used to make the movie)
# the post-processing can be modifed by assigning a depth-weight function depending on the secchi depth. 
# I implemented it for the Uroglena paper, and we can do it later for all lakes

xp_conv *= 1.e-3
yp_conv *= 1.e-3

%matplotlib inline
for qq in range(len(time_plt)):
    # cutoff_date = '2021-09-05T09:30:00.000000000'

    ###############
    for zd in zdata:
        # now = zd.sel(time=np.datetime64(cutoff_date), drop=True)
        now = zd.sel(time=time_plt[qq], drop=True)
        xp = (now.xtrack).to_masked_array().filled()
        xp[xp > 1e9] = np.nan
        yp = now.ytrack.to_masked_array().filled()
        yp[yp > 1e9] = np.nan
        zp = now.ztrack.to_masked_array().filled()
        zp[zp > 35] = np.nan


    ###############   
    """part_no_snap, _, _ = np.histogram2d(xp, yp, bins=[xg_plt[0,:],yg_plt[:,0]])
    part_no_snap = part_no_snap.T
    part_no_snap[part_no_snap<2] = np.nan
    min_val = 2 #np.nanquantile(part_no_snap,0.2)
    part_no_snap[part_no_snap<min_val] = np.nan
    # max_val = np.nanquantile(part_no_snap,0.8)
    

    ############### 
    part_depth_snap = part_no_snap.copy()
    part_depth_snap[:,:] = 0

    for ii in range(len(xp)):
        if ((~np.isnan(xp[ii])) & (~np.isnan(yp[ii]))):
            ind_x = np.searchsorted(0.5*(xg_plt[0,:-1]+xg_plt[0,1:]),xp[ii])
            ind_y = np.searchsorted(0.5*(yg_plt[:-1,0]+yg_plt[1:,0]),yp[ii])
            if ((~np.isnan(zp[ii])) & (zp[ii]<5)):
                part_depth_snap[ind_y,ind_x] = part_depth_snap[ind_y,ind_x]+zp[ii]

    part_depth_snap = part_depth_snap/part_no_snap
    # part_depth_snap[part_depth_snap>60] = np.nan
    # part_no_snap[part_depth_snap>10] = np.nan"""

    fig_size = (16,5)


    f = plt.figure(figsize=fig_size)
    #cmocean.cm.balance
    ax = f.add_subplot(121)
    plt.pcolormesh(xp_conv[:-1,:-1], yp_conv[:-1,:-1],lake_arr,alpha=0.1)
    plt.hold=True
    # SS = plt.pcolormesh(xx_sg,yy_sg,Temp, cmap=cmocean.cm.thermal,shading='flat')
    SS = plt.pcolormesh(xp_conv[:-1,:-1],yp_conv[:-1,:-1],part_no_snap,vmin=np.nanquantile(part_no_snap,0.25), vmax=np.nanquantile(part_no_snap,0.75), cmap='viridis')
#     ax.plot(xs,ys, 'k-', lw=1.5)
#     plt.plot(xp_conv, yp_conv,'b',linewidth=0.1) # use plot, not scatter
#     plt.plot(np.transpose(xp_conv), np.transpose(yp_conv),'b',linewidth=0.1) # add this here

    plt.xticks(fontname=font_feature[0],fontsize=font_feature[2])
    plt.yticks(fontname=font_feature[0],fontsize=font_feature[2])

#     ax.set_xlim(497, 565)
#     ax.set_ylim(115, 155)
    ax.set_aspect('equal')
    ax.set_xlabel("Lat (km CH1903)",fontname=font_feature[0],fontsize=font_feature[1])
    ax.set_ylabel("Lon (km CH1903)",fontname=font_feature[0],fontsize=font_feature[1])
    ax.annotate(str(time_plt[qq].values.astype("datetime64[m]")).replace("T", " "),xy=(0.02, 0.9), xycoords='axes fraction',fontname=font_feature[0],fontsize=font_feature[1])

    cbar = plt.colorbar(fraction=0.04,orientation="horizontal",pad=-0.25,extend='max');
    cbar.ax.set_xticklabels([])
    cbar.ax.set_xticks([])
    cbar.set_label(label='$\mathregular{Particle \ concentration\ [-]}$',family=font_feature[0],size=font_feature[1])


    ax2 = f.add_subplot(122)
    plt.pcolormesh(xp_conv[:-1,:-1], yp_conv[:-1,:-1],lake_arr,alpha=0.1)
    plt.hold=True
    # SS = plt.pcolormesh(xx_sg,yy_sg,Temp, cmap=cmocean.cm.thermal,shading='flat')
    SS = plt.pcolormesh(xp_conv[:-1,:-1],yp_conv[:-1,:-1],part_depth_snap, cmap='jet',vmin=0, vmax=20)
    plt.xticks(fontname=font_feature[0],fontsize=font_feature[2])
    plt.yticks(fontname=font_feature[0],fontsize=font_feature[2])

#     ax2.set_xlim(497, 565)
#     ax2.set_ylim(115, 155)
    ax2.set_aspect('equal')
    ax2.set_xlabel("Lat (km CH1903)",fontname=font_feature[0],fontsize=font_feature[1])
    ax2.set_ylabel("Lon (km CH1903)",fontname=font_feature[0],fontsize=font_feature[1])


    cbar2 = plt.colorbar(fraction=0.04,orientation="horizontal",pad=-0.25,extend='max',format='%:.0f');
    cbar2.ax.set_xticklabels([0,5,10,15,20],fontname=font_feature[0],fontsize=font_feature[2])
    cbar2.set_label(label='$\mathregular{Depth\ [m]}$',family=font_feature[0],size=font_feature[1])
    cbar2.ax.xaxis.set_ticks_position('top')

    plt.savefig(output_path+str(qq).zfill(3)+'_'+str(time_plt[qq].values.astype("datetime64[m]")).replace("T", " ")[0:10]+'H'+str(time_plt[qq].values.astype("datetime64[m]")).replace("T", " ")[11:13]+'.png',dpi=300, bbox_inches = 'tight')

    plt.show()
    plt.close()

NameError: name 'xp_conv' is not defined